In [23]:
import datasets


In [6]:
from datasets import load_dataset

In [7]:
p ='/home/ammar/Desktop/LMU/ADL/data/DRG_huggingface'

In [8]:
# import pandas as pd
# def generate_examples(filepath):
#         """Generate images and labels for splits."""
#         imgfolder = '/home/ammar/Desktop/LMU/ADL/data/C. Diabetic Retinopathy Grading/1. Original Images/a. Training Set'
#         csv_path = '/home/ammar/Desktop/LMU/ADL/data/C. Diabetic Retinopathy Grading/2. Groundtruths/a. DRAC2022_ Diabetic Retinopathy Grading_Training Labels.csv'
#         df= pd.read_csv(csv_path)
#         print(df.shape)
#         for k,v in df.iterrows():
# #             print(v['image name'])
# #             print(v['DR grade'])
# #             print('{}/{}'.format(imgfolder,v['image name']))
#             im = Image.open('{}/{}'.format(imgfolder,v['image name'])).convert('RGB')
# #             break

#             yield v['image name'], {
#                             "image": im,
#                             "label": v['DR grade'],
#                         }

In [5]:
ds = load_dataset(p)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset drg_huggingface downloaded and prepared to /home/ammar/.cache/huggingface/datasets/drg_huggingface/plain_text/1.0.0/c9a31b49604aa94e968ec95199cb3a241a8d852331985077c1a0ae6864083c4b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# loading the dataset
# ds = load_dataset('Maysee/tiny-imagenet', split='valid')

# getting an example


In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 611
    })
})

In [8]:
ds['train'][400]

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024>,
 'label': 2}

In [9]:
ex = ds['train'][400]
print(ex)

# seeing the image
image = ex['img']
image.show()


{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F7F24960340>, 'label': 2}


In [13]:

# getting all the labels
labels = ds['train']['label']
print(labels)


[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 2, 2, 2, 0, 0, 1, 0, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 2, 2, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2, 1, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 2, 2, 1, 1, 2, 1, 0, 1, 

In [15]:
# getting label of our example
# print(labels.int2str(ex['label']))

In [16]:
from transformers import AutoFeatureExtractor

#loading the feature extractor
model_name= "microsoft/swin-large-patch4-window12-384-in22k"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)


In [38]:

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['img']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs
  
# applying transform
prepared_ds = ds.with_transform(transform)

In [18]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 611
    })
})

In [19]:

import torch

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [22]:
from transformers import SwinForImageClassification, Trainer, TrainingArguments

labels = ds['train'].features['label'].names

# initialzing the model
model = SwinForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes = True,
)

/home/ammar/anaconda3/envs/adl/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1648016052946/work/aten/src/ATen/native/TensorShape.cpp:2156.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-large-patch4-window12-384-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1536]) in the checkpoint and torch.Size([3, 1536]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from transformers import Trainer, TrainingArguments

batch_size = 16
# Defining training arguments (set push_to_hub to false if you don't want to upload it to HuggingFace's model hub)
training_args = TrainingArguments(
    f"swin-finetuned-DRG",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
training_args

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
log

In [34]:
# prepared_ds["validation"]

In [35]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 384
}

In [39]:
# Instantiate the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["train"],
    tokenizer=feature_extractor,
)

In [ ]:
# Train and save results
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


***** Running training *****
  Num examples = 611
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 27


In [ ]:

# Evaluate on validation set
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)